<a href="https://colab.research.google.com/github/Ryu4824/code-states/blob/main/n213_discussion_6%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **N213 Regularized Regression**

## 오늘의 목표
- scikit-learn을 이용해 정규화 회귀 모델을 만들어 학습하고 해석할 수 있습니다.
- 정규화 회귀 모델에서 최적의 알파값을 찾을 수 있습니다. 
- 정규화 회귀 모델을 통해 과적합을 해소할 수 있습니다. 

## **개념 Topic**
> 오늘은 정규화 회귀(Regularized Regression)에 대해 배웠습니다. 

- 정규화 회귀를 한 문장으로 요약하여 설명해보세요.
  - `정규화 회귀모델은 선형모델에 규제항을 더해 과적합을 방지하는 방법입니다.` 를 제외한 문장이어야 합니다.
  - 어떤 키워드가 포함되어야할지 키워드 위주로 고민해보세요.
  - 만약 설명이 어렵다면 어느 부분이 이해가 안 되는지 구체적인 질문을 동기들과 나눠보세요.
- 정규화 회귀에 대해 초등학생도 이해할 수 있는 쉬운 비유를 함께 만들어보세요.

  ```
  e.g. 빙판길 사고를 방지하기 위해 자동차 타이어에 체인을 감아 속도를 낮춘다.
  - 빙판길 → 너무 복잡한 모델
  - 사고 위험 → 과적합 발생 가능성
  - 자동차 타이어 → 선형 모델
  - 체인 → 규제항
  - 속도 감소 → 선형모델에서 파라미터 감소
  ```
  - 체크리스트
    - 과적합과 규제항에 대한 비유가 올바른가?
    - 파라미터가 감소한다는 결론이 포함되었는가?
- **Discussion** 표의 `정리` 탭에 답변을 정리하여 적어 주세요. 

- 정규화 회귀란 : 선형회귀 계수에 대한 제약 조건을 추가함으로써 <br>
훈련 모델의 가중치의 크기에 대한 패널티를 부여하여, 복잡도를 줄이고 훈련 모델에 과도하게 최적화되는 현상을 방지하는 방법


클라이밍 할 때 손에 초크 가루를 묻혀 미끄러지지 않게 한다
- 홀드 : 너무 복잡한 모델
- 추락 위험 : 과접합 발생 가능성
- 악력 : 선형 모델
- 초크 가루 : 규제항
- 마찰력 상승 : 선형모델에서 파라미터 감소

## **코딩 Topic**

### **Part.1 : 데이터 준비**
지난 노트에서도 사용했던 인도의 한 도시인 [Begaluru의 집값 데이터](https://www.kaggle.com/datasets/amitabhajoy/bengaluru-house-price-data)를 사용해서 집값을 예측하는 회귀 문제를 풀어보겠습니다.

> **Data Description**

- Area_type : Description of the area
- Availability : When it can be possessed or when it is ready
- Location : Where it is located in Bengaluru
- Size : BHK or Bedrooms
- Society : To which society it belongs
- Total_sqft : Size of the property in sq.ft
- Bath : No. of Bathrooms
- Balcony : No. of the Balcony
- Price : Value of the property in lakhs (Indian Rupee - ₹)


In [ ]:
!pip install category_encoders

In [ ]:
import re
import math
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

#### **1-1. 데이터셋 불러오기 및 전처리**
- 데이터셋을 불러오세요.
- 결측치가 30% 이상인 column은 제거하세요. 
- 결측치인 샘플(row)도 제거 후, index를 재정렬하세요. 
  - 결측치를 모두 제거한 데이터프레임의 shape은 (12710, 8)입니다. 
- 지난 디스커션에서와 마찬가지로 `total_sqft` column을 수치형으로 변환하세요. 

In [ ]:
# 데이터셋 불러오기
import pandas as pd
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/renewal/mldl/House_Data.csv')
df.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [ ]:
# 결측치 확인
df.isna().sum()[df.isna().sum() !=0]/len(df)

location    0.000075
size        0.001201
society     0.413063
bath        0.005480
balcony     0.045721
dtype: float64

In [ ]:
#결측치 30% 이상인 컬럼 제거
df.drop('society', axis=1, inplace=True)

In [ ]:
df.dropna(axis=0, inplace = True)

In [ ]:
df.reset_index(drop=True, inplace = True)

In [ ]:
# 수치로 변환되지 않는 데이터 index 추출
idx = []
for i in range(len(df.total_sqft)):
  try:
    float(df.total_sqft[i])
  except:
    idx.append(i)

In [ ]:
df.drop(idx, inplace = True)

In [ ]:
df.dtypes

area_type        object
availability     object
location         object
size             object
total_sqft       object
bath            float64
balcony         float64
price           float64
dtype: object

In [ ]:
# 수치형으로 변환되지 않는 값의 index에 해당하는 샘플 제거
drop_c = df.columns[df.isna().mean() >= 0.3]
df.drop(drop_c, axis=1, inplace=True)
df

,area_type,availability,location,size,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,1200,2.0,1.0,51.00
...,...,...,...,...,...,...,...,...
13314,Super built-up Area,Ready To Move,Green Glen Layout,3 BHK,1715,3.0,3.0,112.00
13315,Built-up Area,Ready To Move,Whitefield,5 Bedroom,3453,4.0,0.0,231.00
13317,Built-up Area,Ready To Move,Raja Rajeshwari Nagar,2 BHK,1141,2.0,1.0,60.00
13318,Super built-up Area,18-Jun,Padmanabhanagar,4 BHK,4689,4.0,1.0,488.00


In [ ]:
df['total_sqft']  = df['total_sqft'].astype(float)

#### **1-2. 데이터셋 분리**
- 특성 X와 타겟 y를 지정하세요.
  - 타겟은 `price`입니다.
- X와 y 모두 train/test set으로 분리하세요. 
  - train : test 비를 적절히 설정합니다.
  - `random_state`를 설정하여 고정된 결과가 나오도록 해 보세요.
- train set을 train/val set으로 분리하세요.
  - train : val 비를 적절히 설정합니다.
  - `random_state`를 설정하여 고정된 결과가 나오도록 해 보세요.

In [ ]:
#특성 타겟 X, y 지정
target = 'price'
features = ['area_type',	'availability',	'location', 'size',	'total_sqft',	'bath',	'balcony'	]
X = df[features]
y = df[target]

In [ ]:
#데이터 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state = 42)

#### **1-3. Scaling 후 데이터를 확인합니다.**
- 수치형 변수에 `StandardScaler()`를 적용하세요. 

In [ ]:
#수치형 변수 스케일링 조정
from sklearn.preprocessing import StandardScaler

numeric_feats = X_train.dtypes[X_train.dtypes != "object"].index

scaler = StandardScaler()
X_train[numeric_feats] = scaler.fit_transform(X_train[numeric_feats])
X_val[numeric_feats] = scaler.transform(X_val[numeric_feats])
X_test[numeric_feats] = scaler.transform(X_test[numeric_feats])

In [ ]:
#수치형 컬럼 변환 확인
X_train[numeric_feats].describe().T[['mean', 'std']]

,mean,std
total_sqft,2.659549e-18,1.000062
bath,4.432581e-18,1.000062
balcony,0.000000e+00,1.000062


#### **1-4. Encoding 후 데이터를 확인합니다.**
- 범주형 변수에 `OneHotEncoder()`를 적용하세요. 

In [ ]:
# One-Hot encoding
from category_encoders import OneHotEncoder

ohe = OneHotEncoder()

X_train_ohe = ohe.fit_transform(X_train) #train에 fit을 적용하여 틀을 만듬
X_val_ohe = ohe.transform(X_val) #train 틀에 val을 맞춤
X_test_ohe = ohe.transform(X_test) #train 틀에 val을 맞춤

In [ ]:
X_train['size'].unique()

array(['3 BHK', '2 BHK', '1 BHK', '4 Bedroom', '4 BHK', '8 Bedroom',
       '2 Bedroom', '3 Bedroom', '5 Bedroom', '6 Bedroom', '1 Bedroom',
       '8 BHK', '7 Bedroom', '9 Bedroom', '5 BHK', '7 BHK', '13 BHK',
       '6 BHK', '1 RK', '11 BHK', '10 Bedroom', '43 Bedroom',
       '11 Bedroom', '12 Bedroom', '9 BHK', '14 BHK', '27 BHK'],
      dtype=object)

### **Part.2 : 모델 학습 및 평가**

#### **2-1. 기준모델**
- 평균을 사용하여 기준모델을 만들고, $R^2$와 MAE 값을 확인하세요.
  - 기준모델은 train set에 대하여 생성합니다.

In [ ]:
## 기준모델
from sklearn.metrics import r2_score, mean_absolute_error
baseline = [y_train.mean()] * len(y_train)
baseline_r2 = r2_score(y_train, baseline)
baseline_mae = mean_absolute_error(y_train, baseline)
print(baseline_r2, baseline_mae)

0.0 65.17387978949326


In [ ]:
import numpy as np

def print_R2(model, X_train, y_train, X_test, y_test) :

    train_R2 = np.round(model.score(X_train, y_train) , 3)
    test_R2 = np.round(model.score(X_test, y_test),3)
    
    print(f'학습 세트 r2_score : {train_R2}')
    print(f'테스트 세트 r2_score : {test_R2}')
    
    return train_R2, test_R2

In [ ]:
def print_MAE(model, X_train, y_train, X_test, y_test) :
  y_pred_train = model.predict(X_train)
  y_pred_test = model.predict(X_test)
  
  train_mae = mean_absolute_error(y_train, y_pred_train).round(3)
  test_mae = mean_absolute_error(y_test, y_pred_test).round(3)
  
  print(f'학습 세트 MAE : {train_mae}')
  print(f'테스트 세트 MAE : {test_mae}')

  return train_mae, test_mae

#### **2-2. 일반 선형 회귀**
- 일반(다중) 선형 회귀 모델을 학습하고, train 및 val set에 대하여 $R^2$와 MAE 값을 확인하세요.
- 상위 10개의 회귀계수 값을 확인하세요. 

In [ ]:
#OLS 모델
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
ols = LinearRegression()
#학습
ols.fit(X_train_ohe, y_train)

LinearRegression()

In [ ]:
#성능 비교 R2
ols_R2_t, ols_R2_tt, = print_R2(ols, X_train_ohe, y_train, X_val_ohe, y_val)

학습 세트 r2_score : 0.708
테스트 세트 r2_score : -7.011835316379527e+18


In [ ]:
#성능 비교 MAE
ols_MAE_t, ols_MAE_tt, = print_MAE(ols, X_train_ohe, y_train, X_val_ohe, y_val)

학습 세트 MAE : 29.848
테스트 세트 MAE : 32843315345.771


In [ ]:
ols_coef = pd.Series(ols.coef_, X_train_ohe.columns)
ols_coef.sort_values(ascending=False).head(10)

availability_41    8.319967e+12
availability_38    8.319967e+12
availability_34    8.319967e+12
availability_5     8.319967e+12
availability_49    8.319967e+12
availability_62    8.319967e+12
availability_73    8.319967e+12
availability_42    8.319967e+12
availability_66    8.319967e+12
availability_30    8.319967e+12
dtype: float64

#### **2-3. Ridge 회귀**
- `RidgeCV`를 통해 Ridge 회귀 모델을 학습하고, 최적의 alpha 값을 찾으세요.
  - alpha의 범위는 자유롭게 설정해 보세요. 
  - 학습 시간이 너무 길어지지 않도록 cv는 5 이하로 설정하세요. 
- train 및 val set에 대하여 $R^2$와 MAE 값을 구하세요. 
- 상위 10개의 회귀계수 값을 확인하세요.

In [ ]:
from sklearn.linear_model import Ridge, Lasso
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
for alpha in [1, 100.0, 1000.0]:
        
    print(f'Ridge Regression, alpha={alpha}')

    # Ridge 모델 학습
    ridge = Ridge(alpha=alpha)  
    ridge.fit(X_train_ohe, y_train)

    # 성능 확인
    print_R2(ridge, X_train_ohe, y_train, X_val_ohe, y_val)
    print_MAE(ridge, X_train_ohe, y_train, X_val_ohe, y_val)

    # plot coefficients
    # 절대값 상위 10개의 회귀계수
    coefficients = pd.Series(ridge.coef_, X_train_ohe.columns)
    idx = np.abs(coefficients).head(10).index
    plt.figure(figsize=(6, 8))
    coefficients[idx].sort_values().plot.barh()
    plt.show()

In [ ]:
from sklearn.linear_model import RidgeCV, LassoCV

# ridgecv로 학습
alphas = np.arange(1, 100, 10)
ridge = RidgeCV(alphas=alphas, cv=5)
ridge.fit(X_train_ohe, y_train)

print("alpha: ", ridge.alpha_)


# 성능 확인
ridge_train, ridge_val = print_R2(ridge, X_train_ohe, y_train, X_val_ohe, y_val)
ridge_train, ridge_val = print_MAE(ridge, X_train_ohe, y_train, X_val_ohe, y_val)

alpha:  11
학습 세트 r2_score : 0.552
테스트 세트 r2_score : 0.505
학습 세트 MAE : 34.061
테스트 세트 MAE : 35.672


In [ ]:
ridge_coef = pd.Series(ridge.coef_, ridge.feature_names_in_)
ridge_coef.sort_values(ascending=False).head(10)

location_682    290.025699
location_837    198.778399
location_573    176.815815
location_665    160.077325
location_403    142.544416
location_147    124.963384
location_97     123.488815
size_15         121.866567
location_67     113.324072
location_31     108.634924
dtype: float64

#### **2-4. Lasso 회귀**
- `LassoCV`를 통해 Lasso 회귀 모델을 학습하고, 최적의 alpha 값을 찾으세요.
  - alpha의 범위는 자유롭게 설정해 보세요. 
  - 학습 시간이 너무 길어지지 않도록 cv는 5 이하로 설정하세요. 
- train 및 val set에 대하여 $R^2$와 MAE 값을 구하세요. 
- 상위 10개의 회귀계수 값을 확인하세요.

In [ ]:
for alpha in [0.001, 0.01, 1.0, 100]:
        
    print(f'Lasso Regression, alpha={alpha}')

    # Ridge 모델 학습
    lasso = Lasso(alpha=alpha)  
    lasso.fit(X_train_ohe, y_train)

    # 성능 확인
    print_R2(lasso, X_train_ohe, y_train, X_val_ohe, y_val)
    print_MAE(lasso, X_train_ohe, y_train, X_val_ohe, y_val)
    # plot coefficients
    # 절대값 상위 10개의 회귀계수 
    coefficients = pd.Series(lasso.coef_, X_train_ohe.columns)
    idx = np.abs(coefficients).head(10).index
    plt.figure(figsize=(6, 8))
    coefficients[idx].sort_values().plot.barh()
    plt.show()

In [ ]:
from sklearn.linear_model import LassoCV

# lassocv로 학습
alphas = np.arange(10, 200, 10)
lasso = LassoCV(alphas=alphas, cv=5, random_state=42)
lasso.fit(X_train_ohe, y_train)

print("alpha: ", lasso.alpha_)

# 성능 확인
lasso_train, lasso_val, lasso_test = print_R2(lasso, X_train_ohe, y_train, X_val_ohe, y_val)
lasso_train, lasso_val, lasso_test = print_MAE(lasso, X_train_ohe, y_train, X_val_ohe, y_val)

alpha:  10
학습 세트 r2_score : 0.345
테스트 세트 r2_score : 0.411


In [ ]:
print('회귀계수가 0이 아닌 특성의 수:', (lasso.coef_!=0).sum())

lasso_coef = pd.Series(lasso.coef_, ridge.feature_names_in_)
lasso_coef[lasso_coef!=0].sort_values(ascending=False)

#### **2-5. 최종 모델**
- 일반 선형 회귀 / Ridge 회귀 / Lasso 회귀 중 가장 성능이 좋다고 판단되는 모델을 최종 모델로 선택하세요.
- test set에 대하여 최종 모델의 $R^2$와 MAE 값을 구하세요. 

### **Conclusion**
> 오늘 Topic을 수행한 결과를 바탕으로, 다음 사항에 대해 답해 주세요.

1. Ridge 및 Lasso 모델의 성능은 기준모델과 비교하여 어떤가요? 
2. 일반 선형 회귀 모델 학습 결과, 어떤 문제가 발생하였나요?
3. Ridge 및 Lasso 모델의 학습 결과는 일반 선형 회귀 모델과 비교하여 어떤가요?
  - 모델 성능은 어떻게 다른가요?
  - 회귀 계수는 어떻게 다른가요?
4. Ridge 및 Lasso 모델은 일반 선형 회귀 모델의 문제를 어떻게 해결할 수 있는지 설명하세요. 
- **Discussion** 표의 `정리` 탭에 답변을 정리하여 적어 주세요. 

## **심화 Topic(optional)**
1. Ridge와 Lasso 회귀의 차이점을 설명하고 언제 어떤 모델을 사용하는 게 더 좋을지 예시를 들어 설명해보세요.
2. 카테고리가 너무 많은 경우(high cardinality)에 원핫인코딩을 사용하기 적합하지 않은 이유는 무엇일까요?
    - 구체적인 예시를 들어서 설명해보세요. (예: 미국의 51개 주)
    - 이 경우 발생할 수 있는 문제는 무엇이 있을까요?